Make NumPy available:

In [1]:
import numpy as np

## Exercise 07.1 (indexing and timing)

Create two very long NumPy arrays `x` and `y` and sum the arrays using:

1. The NumPy addition syntax, `z = x + y`; and
2. A `for` loop that computes the sum entry-by-entry

Compare the time required for the two approaches for vectors of different lengths (use a very long vector for 
the timing). The values of the array entries are not important for this test. Use `%time` to report the time.

*Hint:* To loop over an array using indices, try a construction like:

In [2]:
x = np.ones(10)
y = np.ones(len(x))
for i in range(len(x)):
    print(x[i]*y[i])

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


#### (1) Add two vectors using built-in addition operator:

In [3]:
x = np.random.rand(10000000)
y = np.random.rand(10000000)
%time z = x+y
#it has to be for REALLY long arrays! otherwise the wall time will show up as zero!

#wall time 46.9ms

Wall time: 47.9 ms


#### (2) Add two vectors using own implementation:

In [4]:

    
%time z = [(x[i]+y[i]) for i in range(len(x))]
#list comprehension woohoo! range starts at 0 so it works

#wall time 6.27s
#vectorisation is so much quicker than looping through the indices for a SUPER long array!!!
#this did not work for arrays of length 10000

def addarray(x,y):
    z = np.zeros(len(x))
    for i in range(len(z)):
        z[i] = x[i]+y[i]
    return z
%time properloop = addarray(x,y) #this is slightly faster than list comprehension- 6.72s here vs 7.3s


Wall time: 8.97 s
Wall time: 9.05 s


### Optional extension: just-in-time (JIT) compilation

You will see a large difference in the time required between your NumPy and 'plain' Python implementations. This is due to Python being an *interpreted* language as opposed to a *compiled* language. A way to speed up plain Python implementions is to convert the interpreted Python code into compiled code. A tool for doing this is [Numba](https://numba.pydata.org/).

Below is an example using Numba and JIT to accelerate a computation:

In [5]:
!pip -q install numba 
import numba
import math

def compute_sine_native(x):
    z = np.zeros(len(x))
    for i in range(len(z)):
        z[i] = math.sin(x[i])
    return z

@numba.jit
def compute_sine_jit(x):
    z = np.zeros(len(x))
    for i in range(len(z)):
        z[i] = math.sin(x[i])
    return z
    
x = np.ones(10000000)
%time z = compute_sine_native(x)
compute_sine_jit(x)
%time z = compute_sine_jit(x)
#Wall time: 5.85 s vs Wall time: 279 ms ----numba.jit makes this a lot faster!!!


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Wall time: 7.73 s
Wall time: 266 ms


In [6]:
#always check whether variable names have been redefined....
x = np.random.rand(10000000)
y = np.random.rand(10000000)
def addarray(x,y):
    z = [(x[i]+y[i]) for i in range(len(x))]
    return z
%time nogit = addarray(x,y)
#6.14s
@numba.jit
def gitaddarray(x,y):
    z = [(x[i]+y[i]) for i in range(len(x))]
    return z
%time withgit = gitaddarray(x,y)
#1.1s
#numba makes this so much faster!
    
    



Wall time: 6.08 s
Wall time: 1.16 s


**Task:** Test if Numba can be used to accelerate your implementation that uses indexing to sum two arrays, and by how much.

## Exercise 07.2 (member functions and slicing)

Anonymised scores (out of 60) for an examination are stored in a NumPy array. Write:

1. A function that takes a NumPy array of the raw scores and returns the scores as percentages, sorted from 
   lowest to highest (try using `scores.sort()`, where `scores` is a NumPy array holding the scores).
1. A function that returns the maximum, minimum and mean of the raw scores as a dictionary with the 
   keys '`min`', '`max`' and '`mean`'. Use the NumPy array functions `min()`, `max()` and `mean()` to do the 
   computation, e.g. `max = scores.max()`.  
   
   Design your function for the min, max and mean to optionally exclude the highest and lowest scores from the 
   computation of the min, max and mean. 
   
   *Hint:* sort the array of scores and use array slicing to exclude
   the first and the last entries.

Use the scores 
```python
scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
```
to test your functions.

In [7]:
def to_percentage_and_sort(scores):
    percentscores = (scores*100/60) #vectorisation
    percentscores.sort() #in place sort
    return percentscores


def statistics(scores, exclude=False): #default argument of exclude set to false but user can specify otherwise
    scores.sort()
    if exclude == True:
        scores = scores[1:-1]
    #numpy.ndarray.max format for these functions
    scoredict = {"min": scores.min(), "max": scores.max(), "mean": scores.mean()}
    
    return scoredict

scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
print(to_percentage_and_sort(scores))
print(statistics(scores))
print(statistics(scores,True))

[13.         40.         58.33333333 70.         96.66666667]
{'min': 7.8, 'max': 58.0, 'mean': 33.36}
{'min': 24.0, 'max': 42.0, 'mean': 33.666666666666664}


In [8]:
scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
assert np.isclose(to_percentage_and_sort(scores), [ 13.0, 40.0, 58.33333333,  70.0, 96.66666667]).all()

s0 = statistics(scores)
assert round(s0["min"] - 7.8, 10) == 0.0
assert round(s0["mean"] - 33.36, 10) == 0.0
assert round(s0["max"] - 58.0, 10) == 0.0

s1 = statistics(scores, True)
assert round(s1["min"] - 24.0, 10) == 0.0
assert round(s1["mean"] - 33.666666666666666667, 10) == 0.0
assert round(s1["max"] - 42.0, 10) == 0.0

## Exercise 07.3 (slicing)

For the two-dimensional array

In [9]:
A = np.array([[4.0, 7.0, -2.43, 67.1],
             [-4.0, 64.0, 54.7, -3.33],
             [2.43, 23.2, 3.64, 4.11],
             [1.2, 2.5, -113.2, 323.22]])
print(A)

[[   4.      7.     -2.43   67.1 ]
 [  -4.     64.     54.7    -3.33]
 [   2.43   23.2     3.64    4.11]
 [   1.2     2.5  -113.2   323.22]]


use array slicing for the below operations, printing the results to the screen to check. Try to use array slicing such that your code would still work if the dimensions of `A` were enlarged.



#### 1. Extract the third column as a 1D array

In [10]:
print(A[:,2])

[  -2.43   54.7     3.64 -113.2 ]


#### 2. Extract the first two rows as a 2D sub-array

In [11]:
print(A[:2,:])

[[ 4.    7.   -2.43 67.1 ]
 [-4.   64.   54.7  -3.33]]


#### 3.  Extract the bottom-right $2 \times 2$ block as a 2D sub-array

In [12]:
print(A[-2:,-2:])

[[   3.64    4.11]
 [-113.2   323.22]]


#### 4. Sum the last column

In [13]:
print(A[:,-1]) #or -1: but that would be a slightly different format
print(A[:,-1].sum())

[ 67.1   -3.33   4.11 323.22]
391.1


#### Compute transpose

Compute the transpose of `A` (search online to find the function/syntax to do this).

In [14]:
print(A.transpose())

[[   4.     -4.      2.43    1.2 ]
 [   7.     64.     23.2     2.5 ]
 [  -2.43   54.7     3.64 -113.2 ]
 [  67.1    -3.33    4.11  323.22]]


## Exercise 07.4 (optional extension)

In a previous exercise you implemented the bisection algorithm to find approximate roots of a mathematical function. Use the SciPy bisection function `optimize.bisect` (http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.bisect.html) to find roots of the mathematical function that was used in the previous exercise. Compare the results computed by SciPy and your program from the earlier exercise, and compare the computational time (using `%time`).

In [15]:
import scipy
from scipy import optimize
#apparently python modules don't always import their sub modules so the code above is needed to import specific modules
#for some reason I need both lines of code for things to work, as opposed to just one of them.

def my_f(x):
    "Evaluate polynomial function"
    return x**3 - 6*x**2 + 4*x + 12
def compute_root(f, x0, x1, tol, max_it):
    "Compute roots of a function using bisection" #in the cell above, we effectively turned the mathematical function into a python function that can be called.
    
    it = 0
    
    x_mid = (x0+x1)/2
    
    f_mid = f(x_mid) #need initial value to start loop
    
    while abs(f_mid) >= tol:
        x_mid = (x0 + x1)/2

    
        f0 = f(x0)
        f_mid = f(x_mid) #need to redfine for every loop
        
        if f0*f_mid<0:
            x1 = x_mid #endpoint redefined as the original midpoint between x0 and x1
        else:
            x0 = x_mid
            
        it +=1
        if it > max_it:
            break
     
    return x_mid, f_mid, it
%time x, f, num_it = compute_root(my_f, x0=3, x1=6, tol=1.0e-6, max_it=1000)
print(x,f, num_it)
#997 microseconds
%time sci_root = scipy.optimize.bisect(my_f, 3, 6, maxiter = 1000)
print(sci_root)

#wall time of 12ms/6.94 etc. vs 0ns
#scipy bisect is much quicker and more accurate, although I couldn't replicate the tolerance setting.
#the tolerance in my original implementation was regarding how close f(x) is to 0 
#whereas for scipy it's how close x is to the actual root 


Wall time: 6.94 ms
4.534070134162903 -7.047073751209609e-07 23
Wall time: 0 ns
4.534070196722951
